## Feature Construction

### What dataset do we use?

In this document, given an entire 2017-2018 season's worth of player data from the The Süper Lig, Turkish professional soccer league (≈1,700,000 data points), we generate input (aligning player trajectories based in role and then setting new features based on spatiotemporal data) and output data in order to build model for predicting ball location over the pitch.
Due to losses in data, some matches have been extracted, so the total number of matches used is 299.

### What is our approach?

Our approach is based on the player's role-assignment, rather than the player's individual data. We're given huge set of player spatiotemportal tracking data. In oreder to build our features set, we should arrange all these data in some particular order, so machine should understand what data it does use. We can not randomly pick player's data and use it in data construction. Therefore, using players individual data for analyzing lacks some important contextual concepts. Football is played by 11 players on each side, but there is always an open door for 3 substitution players. We can not also use players' data those aren't in the game, this should cause a lot of noise in the system. For doing so, we developed the role-based analysis.

As in all games, football also has its standard rules for assigning each player role, such as; the rearmost players are defence players, midfielders are players whose position of play is midway, forwards player are players on the team who play nearest to the opposing. Using this information we can assig role to each player.

1. Define which player's in the game.
2. Generate players' average positions based on their position in the timeline of previous 15 minutes.
3. Each minute update their average position.
4. Scale average positions to the range of (0-105)x(0-68) meters based on their standard deviation of distributional data.
5. Assing role to each player.


Sometimes we lost player's data for short period of time (0-15 seconds), even though he is still in the game (not substituted or set-off). This happens when tagger misses to tag data, the player receives medical treatment out of the pitch, or game is stopped (in this case all players data is null). In these praticualr cases we allply below approach.   


1. Chenck wheter player is still in game or not (not substituted or set-off).
2. If player's in game check wheter you're receiving his data or not.
3. If we receive his data then use it in the feature construction.

### Computing Features For a Single Match

In this section, we compute __*139*__ features in total for a single match. Each feature is computed for each second of the game. Thus, we iterate over the seconds of a match and compute each feature per second.

__Referee features:__ *(5 features)*
1. Cords $(x,y)$ 
2. Speed and its directions both on the x-axis and y-axis $\vec{V}$ 

__Goalkeeper features:__ *(5 features)*
1. Cords $(x,y)$ 
2. Speed and its directions both on the x-axis and y-axis $\vec{V}$ 

__Left&Right Group features:__ *(5 features)*
1. Average coords of gorup $(x,y)$ 
2. Denisty of players distribution $D$ 
3. Average speed and its direction only on the x-axis $\vec{V}_{avrg}$ 

__Center Group features:__ *(14 features)*<br>

Note: ($\vec{V}_{Slow}\leq1.5$,  $ 1.5\lt\vec{V}_{hir}\leq3$,  $3\lt\vec{V}_{sprint}$)
  
1. Average coords of the slow group $(x,y)_{slow}$ 
2. Denisty of players distribution in the slow group $D_{slow}$ 
3. Average speed of the slow group and its direction only on the x-axis $\vec{V}_{slow_{avrg}}$
4. Average coords of the hir group $(x,y)_{hir}$ 
6. Average speed of the hir group and its direction only on the x-axis $\vec{V}_{hir_{avrg}}$
7. Coords of the fastest player in the sprint group $(x,y)_{sprint_{max}}$
8. Speed of the fastest player in the sprint group and its directions both on the x-axis and y-axis $\vec{V}_{sprint_{max}}$

In [20]:
features = [
    
'refX',  'refY', 'refSpeed', 'refDirectX', 'refDirectY',


'homeGoalKeeperX',                  'homeGoalKeeperY',                 'homeGoalKeeperSpeed',        
'homeGoalKeeperDirectX',            'homeGoalKeeperDirectY',

'homeLeftBackAvgX',                 'homeLeftBackAvgY',                'homeLeftBackDenisty',
'homeLeftBackAvgSpeed',             'homeLeftBackAvgDirectX',  

'homeCenterBackSlowAvgX',           'homeCenterBackSlowAvgY',          'homeCenterBackSlowDenisity',          
'homeCenterBackSlowAvgSpeed',       'homeCenterBackSlowAvgDirectX',    'homeCenterBackHirAvgX',        
'homeCenterBackHirAvgY',            'homeCenterBackHirAvgSpeed',       'homeCenterBackHirAvgDirectX', 
'homeCenterBackMaxSprintX',         'homeCenterBackMaxSprintY',        'homeCenterBackMaxSprintSpeed',
'homeCenterBackMaxSprintDirectX',   'homeCenterBackMaxSprintDirectY', 

'homeRightBackAvgX',                'homeRightBackAvgY',               'homeRightBackDenisty',
'homeRightBackAvgSpeed',            'homeRightBackAvgDirectX', 

'homeLeftWingerAvgX',               'homeLeftWingerAvgY',              'homeLeftWingerDenisty',
'homeLeftWingerAvgSpeed',           'homeLeftWingerAvgDirectX',  

'homeCenterMiddleSlowAvgX',         'homeCenterMiddleSlowAvgY',        'homeCenterMiddleSlowDenisity',           
'homeCenterMiddleSlowAvgSpeed',     'homeCenterMiddleSlowAvgDirectX',  'homeCenterMiddleHirAvgX',        
'homeCenterMiddleHirAvgY',          'homeCenterMiddleHirAvgSpeed',     'homeCenterMiddleHirAvgDirectX', 
'homeCenterMiddleMaxSprintX',       'homeCenterMiddleMaxSprintY',      'homeCenterMiddleMaxSprintSpeed',
'homeCenterMiddleMaxSprintDirectX', 'homeCenterMiddleMaxSprintDirectY', 

'homeRightWingerAvgX',              'homeRightWingerAvgY',              'homeRightWingerDenisty',
'homeRightWingerAvgSpeed',          'homeRightWingerAvgDirectX', 

'homeCenterForwardSlowAvgX',        'homeCenterForwardSlowAvgY',        'homeCenterForwardSlowDenisity',           
'homeCenterForwardSlowAvgSpeed',    'homeCenterForwardSlowAvgDirectX',  'homeCenterForwardHirAvgX',        
'homeCenterForwardHirAvgY',         'homeCenterForwardHirAvgSpeed',     'homeCenterForwardHirAvgDirectX', 
'homeCenterForwardMaxSprintX',      'homeCenterForwardMaxSprintY',      'homeCenterForwardMaxSprintSpeed',
'homeCenterForwardMaxSprintDirectX','homeCenterForwardMaxSprintDirectY', 



'awayGoalKeeperX',                  'awayGoalKeeperY',                 'awayGoalKeeperSpeed',        
'awayGoalKeeperDirectX',            'awayGoalKeeperDirectY',

'awayLeftBackAvgX',                 'awayLeftBackAvgY',                'awayLeftBackDenisty',
'awayLeftBackAvgSpeed',             'awayLeftBackAvgDirectX',  

'awayCenterBackSlowAvgX',           'awayCenterBackSlowAvgY',          'awayCenterBackSlowDenisity',          
'awayCenterBackSlowAvgSpeed',       'awayCenterBackSlowAvgDirectX',    'awayCenterBackHirAvgX',        
'awayCenterBackHirAvgY',            'awayCenterBackHirAvgSpeed',       'awayCenterBackHirAvgDirectX', 
'awayCenterBackMaxSprintX',         'awayCenterBackMaxSprintY',        'awayCenterBackMaxSprintSpeed',
'awayCenterBackMaxSprintDirectX',   'awayCenterBackMaxSprintDirectY', 

'awayRightBackAvgX',                'awayRightBackAvgY',               'awayRightBackDenisty',
'awayRightBackAvgSpeed',            'awayRightBackAvgDirectX', 

'awayLeftWingerAvgX',               'awayLeftWingerAvgY',              'awayLeftWingerDenisty',
'awayLeftWingerAvgSpeed',           'awayLeftWingerAvgDirectX',  

'awayCenterMiddleSlowAvgX',         'awayCenterMiddleSlowAvgY',        'awayCenterMiddleSlowDenisity',           
'awayCenterMiddleSlowAvgSpeed',     'awayCenterMiddleSlowAvgDirectX',  'awayCenterMiddleHirAvgX',        
'awayCenterMiddleHirAvgY',          'awayCenterMiddleHirAvgSpeed',     'awayCenterMiddleHirAvgDirectX', 
'awayCenterMiddleMaxSprintX',       'awayCenterMiddleMaxSprintY',      'awayCenterMiddleMaxSprintSpeed',
'awayCenterMiddleMaxSprintDirectX', 'awayCenterMiddleMaxSprintDirectY', 

'awayRightWingerAvgX',              'awayRightWingerAvgY',              'awayRightWingerDenisty',
'awayRightWingerAvgSpeed',          'awayRightWingerAvgDirectX', 

'awayCenterForwardSlowAvgX',        'awayCenterForwardSlowAvgY',        'awayCenterForwardSlowDenisity',          
'awayCenterForwardSlowAvgSpeed',    'awayCenterForwardSlowAvgDirectX',  'awayCenterForwardHirAvgX',        
'awayCenterForwardHirAvgY',         'awayCenterForwardHirAvgSpeed',     'awayCenterForwardHirAvgDirectX', 
'awayCenterForwardMaxSprintX',      'awayCenterForwardMaxSprintY',      'awayCenterForwardMaxSprintSpeed',
'awayCenterForwardMaxSprintDirectX','awayCenterForwardMaxSprintDirectY', 

]

print(len(features))

139


In [21]:
%matplotlib inline
import matplotlib.pyplot as plt
from __future__ import division

import json
import pickle
import pandas as pd
import numpy as np
import math
from scipy.ndimage.interpolation import shift
import operator
import scipy as sc

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler



pd.options.display.max_columns = None
pd.set_option('compute.use_bottleneck', True)
pd.set_option('compute.use_numexpr', True)

### Computation functions

Function for calculating mean of players x,y positions on the pitch

In [22]:
def caluclate_avrg_pos(c_ap_minutes_step, pos_mean, player, pos_count, activity_count):
    avrg_pos_count = 0 
    for c_ap_minutes in range(0,c_ap_minutes_step):
        pos_mean[player][0] += pos_count[player][0][c_ap_minutes]
        pos_mean[player][1] += pos_count[player][1][c_ap_minutes]
        avrg_pos_count += activity_count[player][c_ap_minutes] 
    if avrg_pos_count!=0:
        pos_mean[player][0] /= avrg_pos_count
        pos_mean[player][1] /= avrg_pos_count
    else:
        pos_mean[player][0] = 0 
        pos_mean[player][1] = 0

Function for checking activity over the specific period of time

In [23]:
def caluclate_activity_count(c_ac_minutes_step, player, activity_count):
    
    activity_count_each = 0 
    for c_ac_minutes in range(0, c_ac_minutes_step):
        activity_count_each += activity_count[player][c_ac_minutes] 
    
    if activity_count_each != 0:
        return 1
    else:
        return 0

Function for calculating last number of non-zero activity occurred in last minute

In [24]:
def caluclate_last_nzero_ac_count(player_gorup_len, activity_count):
    
    last_activity_count = 0
    for player_lac_i in range(0, player_gorup_len):
        if activity_count[player_lac_i][1] !=0:
            last_activity_count += 1
            
    return last_activity_count

Function to find the minute that the last non-zero activty occured in

In [25]:
def caluclate_nzero_activity_count(start, end, player, activity_count):
    
    activity_count_each = 0 
    for c_nz_minutes in range(start, end, -1):
        if activity_count[player][c_nz_minutes-1]!=0:
            return c_nz_minutes-1
        
    return -1

Functions for calculating mean of position data of each player as soon as we have all player data. By that way we'll be able to separately calculate each half mean pos data. 

In [26]:
def fist_time_step_mean_data(half, data_persec_t):
    tmp_time_step_t = 0
    
    for t in data_persec_t:

        first_time_step_i = int(t['minute'])*60 + int(t['second'])
        x_pos = t['xpos']
        y_pos = t['ypos']
        team_id_t = int(t['teamId'])
        jersey_number_t = int(t['jerseyNumber'])
        has_ball_teamId_t = int(t['hasballTeamId'])
        math_half = int(t['half'])


        sec = first_time_step_i-(half-1)*45*60
        time_step = math.floor(sec/60)

        if sec==threshold_second:
            break
            
        if tmp_time_step_t == time_step:
            
            tmp_time_step_t += 1
            
            for player_home in range(0, len(home_team_player)):
                home_pos_count_minutes[player_home][0] = (shift(home_pos_count_minutes[player_home][0], -1, cval=0))
                home_pos_count_minutes[player_home][1] = (shift(home_pos_count_minutes[player_home][1], -1, cval=0))
                home_activity_count_minutes[player_home] = (shift(home_activity_count_minutes[player_home], -1, cval=0))
                
            for player_away in range(0, len(away_team_player)):
                away_pos_count_minutes[player_away][0] = (shift(away_pos_count_minutes[player_away][0], -1, cval=0))
                away_pos_count_minutes[player_away][1] = (shift(away_pos_count_minutes[player_away][1], -1, cval=0))
                away_activity_count_minutes[player_away] = (shift(away_activity_count_minutes[player_away], -1, cval=0))

            

        if math_half == half:
            

            if team_id_t == home_team_id:
                
                player_home = home_dict[str(jersey_number_t)]
                if player_positionId['homeTeam'][jersey_number_t] == 1 and has_ball_teamId_t != 0:
                    if x_pos!=0 and y_pos !=0:    
                        home_pos_count_minutes[player_home][0][minutes_step-1] += x_pos
                        home_pos_count_minutes[player_home][1][minutes_step-1] += y_pos
                        home_activity_count_minutes[player_home][minutes_step-1] +=1 

                elif has_ball_teamId_t == away_team_id:
                    if x_pos!=0 and y_pos !=0:    
                        home_pos_count_minutes[player_home][0][minutes_step-1] += x_pos
                        home_pos_count_minutes[player_home][1][minutes_step-1] += y_pos
                        home_activity_count_minutes[player_home][minutes_step-1] +=1 

            elif team_id_t == away_team_id:

                player_away = away_dict[str(jersey_number_t)]
                if player_positionId['awayTeam'][jersey_number_t] == 1 and has_ball_teamId_t != 0:
                    if x_pos!=0 and y_pos!=0:
                        away_pos_count_minutes[player_away][0][minutes_step-1] += x_pos
                        away_pos_count_minutes[player_away][1][minutes_step-1] += y_pos
                        away_activity_count_minutes[player_away][minutes_step-1] +=1  

                elif has_ball_teamId_t == home_team_id:
                    if x_pos!=0 and y_pos!=0:
                        away_pos_count_minutes[player_away][0][minutes_step-1] += x_pos
                        away_pos_count_minutes[player_away][1][minutes_step-1] += y_pos
                        away_activity_count_minutes[player_away][minutes_step-1] +=1  

                        
    return home_activity_count_minutes, away_activity_count_minutes

Function to define pitch index and role

In [27]:
def define_role(x, y, pitch_value, gk_state):
    
    # gk_state==1 := golakeeper
    if gk_state==1:
        return 7
    else:
        for pitch_i in range(0,7):
            if x>=pitch_value[pitch_i][0] and x<pitch_value[pitch_i][1] and y>=pitch_value[pitch_i][2] and y<pitch_value[pitch_i][3]:
                return pitch_i
    return -1

def define_pitch_index(x,y):
    Xi=x//5
    if y<=60:
        Yj = y//5+1
    elif y<=64:
        Yj = 13
    else:
        Yj = 14

    if Xi==21:
        Xi-=1
    index = Xi*14 + Yj

    return index

Function to define features based on pitch index and role

In [28]:
def speed_group(speed):
    if speed<=1.5: # slow Speed (V)
        return 0
    elif speed<=3: # HIR
        return 1
    else: # Sprint (S)
        return 2

def define_direction(delta):
    if deltaX==0:
        return 0
    elif deltaX>0:
        return 1
    else:
        return -1
    
def features_adjust(f_winger_v, f_center_v, f_features_count, team_f_speed_dict, x_data, y_data):

    for f_w_i in range(0,4):
        f_count = int(f_features_count[f_w_i][0])
        if f_count==0:
            f_winger_v[f_w_i][0:5] = 0
        else:
            f_winger_v[f_w_i][0:5] = f_winger_v[f_w_i][0:5] / f_count
            
            deltaX = round(f_winger_v[f_w_i][4],2)
            f_winger_v[f_w_i][4] = define_direction(deltaX)
            
            if len(team_f_speed_dict[f_w_i])==1:
                f_winger_v[f_w_i][2]=1
            elif len(team_f_speed_dict[f_w_i])>1:
                max_distance = []
                for player_a, speed in team_f_speed_dict[f_w_i].items():
                    difX = math.pow(f_winger_v[f_w_i][0]-x_data[player_a][1], 2)
                    difY = math.pow(f_winger_v[f_w_i][1]-y_data[player_a][1], 2)
                    p_distance = math.sqrt(difX+difY)
                    max_distance.append(p_distance)
                    
                
                max_distance = max(max_distance)
                if max_distance<1:
                    max_distance=1
                f_winger_v[f_w_i][2] = 1/max_distance

    for f_a_i in range(4,7):
        slow_group_id=[]
        hir_group_count=0
        for player, speed in team_f_speed_dict[f_a_i].items():
            s_group = speed_group(speed)
            
            if s_group==0:
                f_center_v[f_a_i-4][0] += x_data[player][1]
                f_center_v[f_a_i-4][1] += y_data[player][1]
                f_center_v[f_a_i-4][3] += speed
                f_center_v[f_a_i-4][4] += round((x_data[player][1]-x_data[player][0]),2)
                slow_group_id.append(player)
            elif s_group==1:
                f_center_v[f_a_i-4][5] += x_data[player][1]
                f_center_v[f_a_i-4][6] += y_data[player][1]
                f_center_v[f_a_i-4][7] += speed
                f_center_v[f_a_i-4][8] += round((x_data[player][1]-x_data[player][0]),2)
                hir_group_count +=1
        if len(team_f_speed_dict[f_a_i])!=0 and max(team_f_speed_dict[f_a_i].items(), key=operator.itemgetter(1))[1] > 3:
            max_speed_id = max(team_f_speed_dict[f_a_i].items(), key=operator.itemgetter(1))[0]
            f_center_v[f_a_i-4][9]  = x_data[max_speed_id][1]
            f_center_v[f_a_i-4][10] = y_data[max_speed_id][1]
            f_center_v[f_a_i-4][11] = team_f_speed_dict[f_a_i][max_speed_id]
            f_center_v[f_a_i-4][12] = define_direction(x_data[max_speed_id][1]-x_data[max_speed_id][0])
            f_center_v[f_a_i-4][13] = define_direction(y_data[max_speed_id][1]-y_data[max_speed_id][0])
            
        
        if hir_group_count!=0:
            f_center_v[f_a_i-4][5:9] = f_center_v[f_a_i-4][5:9] / hir_group_count
            
        
        if len(slow_group_id)==1:
            f_center_v[f_a_i-4][0:5] = f_center_v[f_a_i-4][0:5] / len(slow_group_id)
            f_center_v[f_a_i-4][2] = 1
        
        elif len(slow_group_id)>1:
            f_center_v[f_a_i-4][0:5] = f_center_v[f_a_i-4][0:5] / len(slow_group_id)
            max_distance = []
            for player_s in slow_group_id:
                difX = math.pow(f_center_v[f_a_i-4][0]-x_data[player_s][1], 2)
                difY = math.pow(f_center_v[f_a_i-4][1]-y_data[player_s][1], 2)
                p_distance = math.sqrt(difX+difY)
                max_distance.append(p_distance)

            
            max_distance = max(max_distance)
            if max_distance<1:
                max_distance=1
            f_center_v[f_a_i-4][2] = 1/max_distance
            

def feature_generation(team,r,x1,x2,y2,speed,jersey_no,features_v,features_count):
    # homeTema = 0 & awayTeam = 1
    if r>=4 and r<=6:
        features_count[r][0] += 1
        
        if team==0:
            home_f_speed_dict[r][jersey_no]=speed
        else:
            away_f_speed_dict[r][jersey_no]=speed
            
    elif r>=0 and r<=3:
        features_v[r][0] += x2 
        features_v[r][1] += y2
        features_v[r][2] = 0 
        features_v[r][3] += speed 
        features_v[r][4] += round((x2-x1),2)
        
        features_count[r][0] += 1
        
        if team==0:
            home_f_speed_dict[r][jersey_no]=speed
        else:
            away_f_speed_dict[r][jersey_no]=speed
 

Function for sclaing player avrg position to the same size with pitch

In [29]:
def scale_linear_data(rawpoints, high, low):
    mins = np.min(rawpoints, axis=0)
    maxs = np.max(rawpoints, axis=0)
    rng = maxs - mins
    return high - (((high - low) * (maxs - rawpoints)) / rng)

### Computation

In [33]:
with open('../data/general/matches_2017_2018_v1.json') as matches_json:
    matches = json.load(matches_json)

minutes_step = 15 # timeline for calculating avrg pos

role_segment_coord = [[0, 40, 0, 19],    # Left Back 
                      [0, 40, 49, 68],   # Right Back
                      [40, 105, 0, 19],  # Left Winger
                      [40, 105, 49, 68], # Right Winger
                      [0, 35, 19, 49],   # Center Back
                      [35, 75, 19, 49],  # Center Midfielder  
                      [75, 105, 19, 49]] # Center Forward/Attacking Middle



# seeting feature set header
features_set_header = ''
for f in features:
    features_set_header += f + ',' 
    
features_set_header += 'pitch_index'
    
    
# position group count set header
group_count_header = 'matchId, half, min, sec'

g_count_header = ['homeLeftBack',   'homeCenterBack',   'homeRightBack', 
                  'homeLeftWinger', 'homeCenterMiddle', 'homeRightWinger', 
                  'homeCenterForward', 
                  
                  'awayLeftBack',   'awayCenterBack',   'awayRightBack', 
                  'awayLeftWinger', 'awayCenterMiddle', 'awayRightWinger', 
                  'awayCenterForward']

for g in g_count_header:
    group_count_header += ',' + g
    
    
for match_data in matches:

    
    id = int(match_data['id'])
#     if id!=60617 and id!=60634 and id!=60696 and id!=60706 and id!=60821 and id!=60845 and id!=116001326:
    if id==60565:
    
        print("\nMatch id:" + str(id))

        # defining 3 distinct id in game
        referee_id = 0
        home_team_id = int(match_data['homeId']) 
        away_team_id = int(match_data['awayId']) 

        # home and away player list
        home_team_player = []
        home_goal_keeper = []
        away_team_player = []
        away_goal_keeper = []
        
        
        ball_index_state=1
        
        # all data
        all_data = []
        all_data_count = []

        # home_nonzero
        home_player_count_pitch = []
        home_prev_count = 11

        # home_nonzero
        away_player_count_pitch = []
        away_prev_count = 11

        # player dicts
        home_dict = {} 
        away_dict = {}
        
        # features vectors
        home_wingers_fd = np.zeros(shape=(4, 5), dtype=np.float)
        home_centers_fd = np.zeros(shape=(3, 14), dtype=np.float)
        
        away_wingers_fd = np.zeros(shape=(4, 5), dtype=np.float)
        away_centers_fd = np.zeros(shape=(3, 14), dtype=np.float)
     
        
        # features vectors count
        home_feature_count = np.zeros(shape=(7, 1), dtype=np.float)
        away_feature_count = np.zeros(shape=(7, 1), dtype=np.float)
        
        #features speed dict
        home_f_speed_dict = {0:{}, 1:{}, 2:{}, 3:{}, 4:{}, 5:{}, 6:{}}
        away_f_speed_dict = {0:{}, 1:{}, 2:{}, 3:{}, 4:{}, 5:{}, 6:{}}
        
        # features for Refreee and GKs
        refree_features = np.zeros(shape=(1,5), dtype=np.float)
        GK_home_features = np.zeros(shape=(1,5), dtype=np.float)
        GK_away_features = np.zeros(shape=(1,5), dtype=np.float)
        
        # ball position
        ball_pos_index = np.zeros(shape=(1), dtype=np.int)

        # game duration
        whole_game_duration = 0

        # players position ID 
        player_positionId = {'homeTeam' : {}, 'awayTeam' : {}}

        with open('../data/match_' + str(id) + '/roster_data_' + str(id) + '.json', 'r') as file:
            match_squad = json.load(file)

        for roaster in match_squad:
            if roaster['teamId'] == home_team_id:
                home_team_player.append(str(roaster['jerseyNumber']))
                player_positionId['homeTeam'][roaster['jerseyNumber']] = roaster['positionId'] 
                if roaster['positionId'] == 1:
                    home_goal_keeper.append(str(roaster['jerseyNumber']))


            elif roaster['teamId'] == away_team_id:
                away_team_player.append(str(roaster['jerseyNumber']))
                player_positionId['awayTeam'][roaster['jerseyNumber']] = roaster['positionId']
                if roaster['positionId'] == 1:
                    away_goal_keeper.append(str(roaster['jerseyNumber']))


        # home players dict data
        home_team_player.sort(key=int)
        home_goal_keeper.sort(key=int)
        home_val = np.arange(0, len(home_team_player))
        home_dict = dict(zip(home_team_player, home_val))
        home_dict_reverse = dict(zip(home_val, home_team_player))


        # away players dict data
        away_team_player.sort(key=int)
        away_goal_keeper.sort(key=int)
        away_val = np.arange(0, len(away_team_player))
        away_dict = dict(zip(away_team_player, away_val))
        away_dict_reverse = dict(zip(away_val, away_team_player))

        # home and away team players' status[0: not playing, 1: playing, 2-left the game], enter status[0: enter 1st half, 1: enter second half],
        # enter time, and duration, red card
        home_dur_in_out = np.zeros(shape=(len(home_team_player), 5), dtype=np.float)
        away_dur_in_out = np.zeros(shape=(len(away_team_player), 5), dtype=np.float)


        with open('../data/match_' + str(id) + '/players_data_' + str(id) + '.json', 'r') as file:
            player_data = json.load(file)


        for player in player_data:
            jersey_number = player['jerseyNumber']
            if player['teamId'] == home_team_id:
                home_dur_in_out[home_dict[str(jersey_number)]][3] = player['duration']
                home_dur_in_out[home_dict[str(jersey_number)]][4] = player['redCard']

            elif player['teamId'] == away_team_id:
                away_dur_in_out[away_dict[str(jersey_number)]][3] = player['duration']
                away_dur_in_out[away_dict[str(jersey_number)]][4] = player['redCard']

        # home and away team player pos and activity count: 0-xpos, 1-ypos
        home_pos_count = np.zeros(shape=(len(home_team_player), 2), dtype=np.float)
        home_activity_count = np.zeros(shape=(len(home_team_player), 1), dtype=np.int)
        home_activity_count_all = np.zeros(shape=(len(home_team_player), 2), dtype=np.int)
        home_act_nozero = [0,0] # number non-zero activity_count_all


        away_pos_count = np.zeros(shape=(len(away_team_player), 2), dtype=np.float)
        away_activity_count = np.zeros(shape=(len(away_team_player), 1), dtype=np.int)
        away_activity_count_all = np.zeros(shape=(len(away_team_player), 2), dtype=np.int)
        away_act_nozero = [0,0] # number non-zero activity_count_all
        
        # team positions
        home_team_role =  np.zeros(shape=(len(home_team_player), 1), dtype=np.int)
        away_team_role =  np.zeros(shape=(len(away_team_player), 1), dtype=np.int)


        # home and away team player count data for 10 min: 0-xpos, 1-ypos / 0-1min, 1-2min... 9-10min 
        home_pos_count_minutes = np.zeros(shape=(len(home_team_player), 2, minutes_step), dtype=np.float)
        home_activity_count_minutes = np.zeros(shape=(len(home_team_player), minutes_step), dtype=np.int)

        home_pos_count_minutes2 = np.zeros(shape=(len(home_team_player), 2, minutes_step), dtype=np.float)
        home_activity_count_minutes2 = np.zeros(shape=(len(home_team_player), minutes_step), dtype=np.int)

        away_pos_count_minutes = np.zeros(shape=(len(away_team_player), 2, minutes_step), dtype=np.float)
        away_activity_count_minutes = np.zeros(shape=(len(away_team_player), minutes_step), dtype=np.int)

        away_pos_count_minutes2 = np.zeros(shape=(len(away_team_player), 2, minutes_step), dtype=np.float)
        away_activity_count_minutes = np.zeros(shape=(len(away_team_player), minutes_step), dtype=np.int)


        # home and away team mean pos data: 0-xpos, 1-ypos
        home_pos_mean = np.zeros(shape=(len(home_team_player), 2), dtype=np.float)
        away_pos_mean = np.zeros(shape=(len(away_team_player), 2), dtype=np.float)


        # players's and refree's previous and next x&y data
        home_x_data = np.zeros(shape=(len(home_team_player),2), dtype=np.float)
        home_y_data = np.zeros(shape=(len(home_team_player),2), dtype=np.float)
        
        away_x_data = np.zeros(shape=(len(away_team_player),2), dtype=np.float)
        away_y_data = np.zeros(shape=(len(away_team_player),2), dtype=np.float)
        
        referee_xy_data = [[0,0],[0,0]]

        with open('../data/match_' + str(id) + '/per_sec_data_' + str(id) + '.json', 'r') as file:
            data_persec = json.load(file)

        home_pos_count_minutes = np.zeros(shape=(len(home_team_player), 2, minutes_step), dtype=np.float)
        home_activity_count_minutes = np.zeros(shape=(len(home_team_player), minutes_step), dtype=np.int)

        away_pos_count_minutes = np.zeros(shape=(len(away_team_player), 2, minutes_step), dtype=np.float)
        away_activity_count_minutes = np.zeros(shape=(len(away_team_player), minutes_step), dtype=np.int)

        # create first average positon data for assigning player position at the 1st half
        # temporary t
        
        home_activity_count_all[0: len(home_team_player)] = 0
        away_activity_count_all[0: len(away_team_player)] = 0

        
        tmp_t = 0
        tmp_t_2 = 0
        shift_times_right = 0
        
        # threshol second:
        threshold_second = 60

        # 1st half duration
        first_half_duration = 0

        home_nonzero = 11
        away_nonzero = 11

        prev_home_nonzero = 11
        prev_away_nonzero = 11


        # temporary half
        tmp_half = 1

        c_activity_value = 0
        c_home_activity_each = 0
        c_away_activity_each = 0

        while c_activity_value==0:

            c_home_activity_each = 0
            c_away_activity_each = 0

            a,b = fist_time_step_mean_data(1, data_persec)


            for player_home in range(0, len(home_team_player)):
                res = caluclate_activity_count(minutes_step, player_home, a)
                c_home_activity_each += res

            for player_away in range(0, len(away_team_player)):
                res = caluclate_activity_count(minutes_step, player_away, b)
                c_away_activity_each += res

            if c_home_activity_each == 11 and c_away_activity_each == 11:
                c_activity_value = 1

            threshold_second += 60  

        threshold_second -= 1
        tmp_t_2 = threshold_second 
        
#         print("1st half start: " + str(threshold_second) + " sec")
        
        
        
        for d in data_persec:

            match_half_t = int(d['half'])

            # create first average positon data for assigning player at the 2nd half
            if match_half_t != tmp_half and match_half_t==2:
                tmp_half = match_half_t
                shift_times_right = 0

                tmp_t = 45*60

                activity_home_nonzero = 0
                activity_away_nonzero = 0
                home_break_subs = []
                away_break_subs = []
                home_break_subs_after = []
                away_break_subs_after = []

                for player_home in range(0, len(home_team_player)):
                    if home_pos_mean[player_home][0] !=0:
                            activity_home_nonzero += 1
                            home_break_subs.append(1)
                    else:
                        home_break_subs.append(0)


                for player_away in range(0, len(away_team_player)):
                    if away_pos_mean[player_away][0] !=0:
                            activity_away_nonzero += 1
                            away_break_subs.append(1)
                    else:
                        away_break_subs.append(0)

                home_nonzero = activity_home_nonzero
                away_nonzero = activity_away_nonzero


                # clear home team data
                home_pos_count[0: len(home_team_player)] = 0
                home_activity_count[0: len(home_team_player)] = 0 
                home_activity_count_all[0: len(home_team_player)] = 0
                home_act_nozero = [0,0]
                home_pos_count_minutes[0: len(home_team_player)] = 0
                home_activity_count_minutes[0: len(home_team_player)] = 0
                home_pos_mean[0: len(home_team_player)] = 0

                # clear away team data
                away_pos_count[0: len(away_team_player)] = 0
                away_activity_count[0: len(away_team_player)] = 0 
                away_activity_count_all[0: len(away_team_player)] = 0
                away_act_nozero = [0,0]
                away_pos_count_minutes[0: len(away_team_player)] = 0
                away_activity_count_minutes[0: len(away_team_player)] = 0
                away_pos_mean[0: len(away_team_player)] = 0
                
 
                # features vectors
                home_wingers_fd[:]=0
                home_centers_fd[:]=0
                away_wingers_fd[:]=0
                away_centers_fd[:]=0

                # features vectors count
                home_feature_count[:]=0
                away_feature_count[:]=0
                
                #features speed dict
                home_f_speed_dict = {0:{}, 1:{}, 2:{}, 3:{}, 4:{}, 5:{}, 6:{}}
                away_f_speed_dict = {0:{}, 1:{}, 2:{}, 3:{}, 4:{}, 5:{}, 6:{}}
                
                # index 
                ball_pos_index[0] = 0
     
                

                refree_features[:]=0
                GK_home_features[:]=0
                GK_away_features[:]=0
                
                
                # previus and next x data
                home_x_data[:]=0
                away_x_data[:]=0
                referee_xy_data = [[0,0],[0,0]]

                threshold_second = 60

                c_activity_value = 0
                c_home_activity_each = 0
                c_away_activity_each = 0

                while c_activity_value==0:

                    c_home_activity_each = 0
                    c_away_activity_each = 0

                    home_break_subs_after = []
                    away_break_subs_after = []

                    a,b = fist_time_step_mean_data(2, data_persec)

                    for player_home in range(0, len(home_team_player)):
                        res = caluclate_activity_count(minutes_step, player_home, a)
                        c_home_activity_each += res
                        home_break_subs_after.append(res)

                    for player_away in range(0, len(away_team_player)):
                        res = caluclate_activity_count(minutes_step, player_away, b)
                        c_away_activity_each += res
                        away_break_subs_after.append(res)


                    if c_home_activity_each == activity_home_nonzero and c_away_activity_each == activity_away_nonzero:
                        c_activity_value = 1

                    threshold_second += 60


                for player_home_2 in range(0, len(home_team_player)):
                    if home_break_subs_after[player_home_2]==0 and home_break_subs[player_home_2]==1:
                        home_dur_in_out[player_home_2][0] = 2 # menas goal keepr has left the game

                for player_away_2 in range(0, len(away_team_player)):
                    if away_break_subs_after[player_away_2]==0 and away_break_subs[player_away_2]==1:
                        away_dur_in_out[player_away_2][0] = 2 # menas goal keepr has left the game

                threshold_second -= 1
                tmp_t_2 = threshold_second 

            i = int(d['minute'])*60 + int(d['second'])
            x_pos = round(d['xpos'],2)
            y_pos = round(d['ypos'],2)
            team_id_t = int(d['teamId'])
            speed_t = round(d['speed'],2)
            jersey_number_t = int(d['jerseyNumber'])
            has_ball_teamId_t = int(d['hasballTeamId'])
            has_ball_jerseyId_t = int(d['hasballJerseyNumber'])


            # find out how long did 1st half last
            if match_half_t == 1:
                first_half_duration = i


            if i%60==59:

                if team_id_t == home_team_id:
                    player = home_dict[str(jersey_number_t)]
                    home_activity_count_all[player][1] = 1

                elif team_id_t == away_team_id:
                    player = away_dict[str(jersey_number_t)]
                    away_activity_count_all[player][1] = 1
      
            if i%60!=0:    

                if team_id_t == home_team_id:

                    if home_dur_in_out[home_dict[str(jersey_number_t)]][0] == 0:
                        home_dur_in_out[home_dict[str(jersey_number_t)]][0] = 1 # indicates that a player is in game 
                        home_dur_in_out[home_dict[str(jersey_number_t)]][1] = match_half_t # enter half
                        home_dur_in_out[home_dict[str(jersey_number_t)]][2] = i  # enter time


                        plyr_position = player_positionId['homeTeam'][jersey_number_t]
                        player = home_dict[str(jersey_number_t)]
                        home_pos_count_minutes[player][1][minutes_step-1] += 34
                        home_activity_count_minutes[player][minutes_step-1] +=1


                        if plyr_position == 1:
                            home_pos_count_minutes[player][0][minutes_step-1] += 10
                        elif plyr_position == 2:
                            home_pos_count_minutes[player][0][minutes_step-1] += 38
                        elif plyr_position == 3:
                            home_pos_count_minutes[player][0][minutes_step-1] += 58
                        else:
                            home_pos_count_minutes[player][0][minutes_step-1] += 78

                    if i<((home_dur_in_out[home_dict[str(jersey_number_t)]][2])+60) and home_dur_in_out[home_dict[str(jersey_number_t)]][0] == 1:
                        if x_pos!=0 and y_pos!=0:
                            home_pos_count[home_dict[str(jersey_number_t)]][0] += x_pos
                            home_pos_count[home_dict[str(jersey_number_t)]][1] += y_pos
                            home_activity_count[home_dict[str(jersey_number_t)]] +=1 

                    elif player_positionId['homeTeam'][jersey_number_t] == 1 and has_ball_teamId_t != 0:
                        if x_pos!=0 and y_pos!=0:
                            home_pos_count[home_dict[str(jersey_number_t)]][0] += x_pos
                            home_pos_count[home_dict[str(jersey_number_t)]][1] += y_pos
                            home_activity_count[home_dict[str(jersey_number_t)]] +=1

                    elif has_ball_teamId_t == away_team_id:
                        if x_pos!=0 and y_pos!=0:
                            home_pos_count[home_dict[str(jersey_number_t)]][0] += x_pos
                            home_pos_count[home_dict[str(jersey_number_t)]][1] += y_pos
                            home_activity_count[home_dict[str(jersey_number_t)]] +=1 

                elif team_id_t == away_team_id:

                    if away_dur_in_out[away_dict[str(jersey_number_t)]][0] == 0:
                        away_dur_in_out[away_dict[str(jersey_number_t)]][0] = 1 # indicates that a player is in game
                        away_dur_in_out[away_dict[str(jersey_number_t)]][1] = match_half_t # enter half 
                        away_dur_in_out[away_dict[str(jersey_number_t)]][2] = i #enter time

                        plyr_position = player_positionId['awayTeam'][jersey_number_t]
                        player = away_dict[str(jersey_number_t)]
                        away_pos_count_minutes[player][1][minutes_step-1] += 34
                        away_activity_count_minutes[player][minutes_step-1] +=1


                        if plyr_position == 1:
                            away_pos_count_minutes[player][0][minutes_step-1] += 10
                        elif plyr_position == 2:
                            away_pos_count_minutes[player][0][minutes_step-1] += 38
                        elif plyr_position == 3:
                            away_pos_count_minutes[player][0][minutes_step-1] += 58
                        else:
                            away_pos_count_minutes[player][0][minutes_step-1] += 78


                    if i<((away_dur_in_out[away_dict[str(jersey_number_t)]][2])+60) and away_dur_in_out[away_dict[str(jersey_number_t)]][0] == 1:
                        if x_pos!=0 and y_pos!=0:
                            away_pos_count[away_dict[str(jersey_number_t)]][0] += x_pos
                            away_pos_count[away_dict[str(jersey_number_t)]][1] += y_pos
                            away_activity_count[away_dict[str(jersey_number_t)]] +=1

                    elif player_positionId['awayTeam'][jersey_number_t] == 1 and has_ball_teamId_t != 0:
                        away_pos_count[away_dict[str(jersey_number_t)]][0] += x_pos
                        away_pos_count[away_dict[str(jersey_number_t)]][1] += y_pos
                        away_activity_count[away_dict[str(jersey_number_t)]] +=1

                    elif has_ball_teamId_t == home_team_id:
                         if x_pos!=0 and y_pos!=0:
                            away_pos_count[away_dict[str(jersey_number_t)]][0] += x_pos
                            away_pos_count[away_dict[str(jersey_number_t)]][1] += y_pos
                            away_activity_count[away_dict[str(jersey_number_t)]] +=1 

             
            # shifing last index of 15min arry (period of 15min) with last min data
            # and calculate mena position of each player each min based on previus 10 min
            if i%60 == 0 and tmp_t != i+1 and (i-(match_half_t-1)*45*60)>threshold_second:


                tmp_t = i+1

                prev_home_nonzero = home_nonzero
                home_nonzero = 0

                prev_away_nonzero = away_nonzero
                away_nonzero = 0


                if shift_times_right !=0:
                    for player_home in range(0, len(home_team_player)):

                        home_pos_count_minutes[player_home][0] = (shift(home_pos_count_minutes[player_home][0], -1, cval=0))
                        home_pos_count_minutes[player_home][1] = (shift(home_pos_count_minutes[player_home][1], -1, cval=0))
                        home_activity_count_minutes[player_home] = (shift(home_activity_count_minutes[player_home], -1, cval=0))

                        home_pos_count_minutes[player_home][0][minutes_step-1] = home_pos_count[player_home][0]
                        home_pos_count_minutes[player_home][1][minutes_step-1] = home_pos_count[player_home][1]
                        home_activity_count_minutes[player_home][minutes_step-1] = home_activity_count[player_home]


                    for player_away in range(0, len(away_team_player)):

                        away_pos_count_minutes[player_away][0] = (shift(away_pos_count_minutes[player_away][0], -1, cval=0))
                        away_pos_count_minutes[player_away][1] = (shift(away_pos_count_minutes[player_away][1], -1, cval=0))
                        away_activity_count_minutes[player_away] = (shift(away_activity_count_minutes[player_away], -1, cval=0))

                        away_pos_count_minutes[player_away][0][minutes_step-1] = away_pos_count[player_away][0]
                        away_pos_count_minutes[player_away][1][minutes_step-1] = away_pos_count[player_away][1]
                        away_activity_count_minutes[player_away][minutes_step-1] = away_activity_count[player_away]


                shift_times_right = 1


                # clear home team
                home_pos_count[0: len(home_team_player)] = 0
                home_activity_count[0: len(home_team_player)] = 0

                # clear away team
                away_pos_count[0: len(away_team_player)] = 0
                away_activity_count[0: len(away_team_player)] = 0


                home_pos_mean[0:len(home_team_player)] = 0
                home_act_nozero = [0,0]

                away_pos_mean[0:len(away_team_player)] = 0
                away_act_nozero = [0,0]

                for player_home in range(0, len(home_team_player)):
                    jersey_n = int(home_dict_reverse[player_home])
                    if player_positionId['homeTeam'][jersey_n] == 1 and home_dur_in_out[player_home][0]==1 and home_activity_count_all[player_home][1] == 0:
                        action_status_exit = 0
                        
                        if home_dur_in_out[player_home][4]==1:
                            enter_half = home_dur_in_out[player_home][1]
                            enter_time = home_dur_in_out[player_home][2]
                            p_game_play_dur = home_dur_in_out[player_home][3]

                            h = match_half_t
                            f_h_d= first_half_duration

                            if enter_half == 1:
                                current_play_dur = math.floor( ((f_h_d - enter_time) + (i-45*60)*(h-1) )/60 )
                            elif enter_half == 2:
                                current_play_dur = math.floor( (i - enter_time)/60 )

                            if current_play_dur < p_game_play_dur:


                                for gk_i in home_goal_keeper:
                                    if home_activity_count_all[home_dict[str(gk_i)]][1] !=0:
                                        home_dur_in_out[player_home][0] = 2 # menas goal keepr has left the game
                                        home_pos_mean[player_home][0] = 0 
                                        home_pos_mean[player_home][1] = 0
                                        action_status_exit = 1
                        else:
                            for gk_i in home_goal_keeper:
                                if home_activity_count_all[home_dict[str(gk_i)]][1] !=0:
                                    home_dur_in_out[player_home][0] = 2 # menas goal keepr has left the game
                                    home_pos_mean[player_home][0] = 0 
                                    home_pos_mean[player_home][1] = 0
                                    action_status_exit = 1
                            

                        if action_status_exit == 0:
                            home_activity_count_all[player_home][1] = 1

                    if (home_activity_count_all[player_home][0]==1):
                        home_act_nozero[0] += 1

                    if (home_activity_count_all[player_home][1]==1):
                        home_act_nozero[1] += 1
                        
                

                for player_away in range(0, len(away_team_player)):
                    jersey_n = int(away_dict_reverse[player_away])
                    if player_positionId['awayTeam'][jersey_n] == 1 and away_dur_in_out[player_away][0]==1 and away_activity_count_all[player_away][1] == 0:
                        action_status_exit = 0
                        
                        if away_dur_in_out[player_away][4]==1:

                            # player current and whole game duration durng match
                            enter_half = away_dur_in_out[player_away][1]
                            enter_time = away_dur_in_out[player_away][2]
                            p_game_play_dur = away_dur_in_out[player_away][3]

                            h = match_half_t
                            f_h_d= first_half_duration

                            if enter_half == 1:
                                current_play_dur = math.floor( ((f_h_d - enter_time) + (i-45*60)*(h-1) )/60 )
                            elif enter_half == 2:
                                current_play_dur = math.floor( (i - enter_time)/60 )

                            if current_play_dur < p_game_play_dur:

                                for gk_i in away_goal_keeper:
                                    if away_activity_count_all[away_dict[str(gk_i)]][1] !=0:
                                        away_dur_in_out[player_away][0] = 2 # menas goal keepr has left the game
                                        away_pos_mean[player_away][0] = 0 
                                        away_pos_mean[player_away][1] = 0
                                        action_status_exit = 1
                        else:
                            for gk_i in away_goal_keeper:
                                if away_activity_count_all[away_dict[str(gk_i)]][1] !=0:
                                    away_dur_in_out[player_away][0] = 2 # menas goal keepr has left the game
                                    away_pos_mean[player_away][0] = 0 
                                    away_pos_mean[player_away][1] = 0
                                    action_status_exit = 1
                            
                        if action_status_exit == 0:
                            away_activity_count_all[player_away][1] = 1


                    if (away_activity_count_all[player_away][0]!=0):
                        away_act_nozero[0] += 1

                    if (away_activity_count_all[player_away][1]!=0):
                        away_act_nozero[1] += 1

                home_subs_count_check = 0
                home_subs_count_id = 0
                home_2subs_array_in = []
                home_2subs_array_out = []


                for player_home in range(0, len(home_team_player)):
                  

                    if home_activity_count_all[player_home][1] != 0 and home_dur_in_out[player_home][0]==1:

                        caluclate_avrg_pos(minutes_step, home_pos_mean, player_home, home_pos_count_minutes, 
                                           home_activity_count_minutes)

                        if home_act_nozero[0]-home_act_nozero[1]<0 and home_act_nozero[0]!=0 and home_activity_count_all[player_home][0] == 0:
                            home_pos_mean[player_home][0] = 0 
                            home_pos_mean[player_home][1] = 0
                            home_activity_count_all[player_home][1] = 0


                    elif home_activity_count_all[player_home][1] == 0 and home_dur_in_out[player_home][0]==1 and home_dur_in_out[player_home][4]==1: 
                        # player current and whole game duration durng match


                        enter_half = home_dur_in_out[player_home][1]
                        enter_time = home_dur_in_out[player_home][2]
                        p_game_play_dur = home_dur_in_out[player_home][3]

                        h = match_half_t
                        f_h_d= first_half_duration

                        if enter_half == 1:
                            current_play_dur = math.floor( ((f_h_d - enter_time) + (i-45*60)*(h-1) )/60 )
                        elif enter_half == 2:
                            current_play_dur = math.floor( (i - enter_time)/60 )

                        if current_play_dur < p_game_play_dur:
                            caluclate_avrg_pos(minutes_step, home_pos_mean, player_home, home_pos_count_minutes, 
                                           home_activity_count_minutes)
                            home_activity_count_all[player_home][1] = 1
                        else:
                            home_dur_in_out[player_home][0] = 2
                            home_pos_mean[player_home][0] = 0 
                            home_pos_mean[player_home][1] = 0


                    elif home_activity_count_all[player_home][1] == 0 and home_dur_in_out[player_home][0]==1:

                        if (home_act_nozero[0] == home_act_nozero[1]):
                            home_dur_in_out[player_home][0]=2
                            home_pos_mean[player_home][0] = 0 
                            home_pos_mean[player_home][1] = 0

                        elif home_act_nozero[0] > home_act_nozero[1]:
                            caluclate_avrg_pos(minutes_step, home_pos_mean, player_home, home_pos_count_minutes, 
                                           home_activity_count_minutes)
                            home_activity_count_all[player_home][1] = 1
                            home_subs_count_check += 1
                            home_2subs_array_out.append(player_home)
                            home_subs_count_id = player_home

                    else:

                        home_pos_mean[player_home][0] = 0 
                        home_pos_mean[player_home][1] = 0


                for player_home in range(0, len(home_team_player)):
                    if home_subs_count_check >=1 and (home_act_nozero[0]-home_act_nozero[1]==1):

                        d = caluclate_nzero_activity_count(minutes_step, 0, home_subs_count_id, home_activity_count_minutes)
                        if home_activity_count_all[player_home][0] == 0 and home_activity_count_all[player_home][1] == 1:
                            
                            if d==14:
                                d-=2
                            result = caluclate_nzero_activity_count(minutes_step, d+1, player_home, home_activity_count_minutes)
                            if result != -1:
                                home_2subs_array_in.append(player_home)

                    home_activity_count_all[player_home][0] = home_activity_count_all[player_home][1]
                    home_activity_count_all[player_home][1] = 0



                # number of last non-zero activties
                home_last_nz_act_count = caluclate_last_nzero_ac_count(len(home_team_player), home_activity_count_all)



                if home_last_nz_act_count == prev_home_nonzero and len(home_2subs_array_out)==2:
                    for home_2subs_i in range(0,2):
                        player = home_2subs_array_out[home_2subs_i]
                        if home_activity_count_minutes[player][minutes_step-1] == 0:
                            home_dur_in_out[player][0] = 2
                            home_activity_count_all[player][0] = 0
                            home_pos_mean[player][0] = 0
                            home_pos_mean[player][1] = 0


                if len(home_2subs_array_out)>=2:
                    for home_2subs_i in range(0,2):
                        player = home_2subs_array_out[home_2subs_i]
                        home_activity_count_all[player][0] = 1
                        caluclate_avrg_pos(minutes_step, home_pos_mean, player_home, home_pos_count_minutes, 
                                           home_activity_count_minutes)


                if len(home_2subs_array_in) == 2 and len(home_2subs_array_out)==2:
                    for home_2subs_i in range(0,2):
                        player = home_2subs_array_out[home_2subs_i]
                        home_dur_in_out[player][0] = 2
                        home_activity_count_all[player][0] = 0
                        home_pos_mean[player][0] = 0
                        home_pos_mean[player][1] = 0

                if len(home_2subs_array_in) == 1 and len(home_2subs_array_out)>=2:
                    player = home_2subs_array_in[0]
                    home_activity_count_all[player][0] = 0
                    home_pos_mean[player][0] = 0
                    home_pos_mean[player][1] = 0


                if home_act_nozero[0]-home_act_nozero[1]==1 and home_subs_count_check==1 and len(home_2subs_array_in) == 1:
                    player_out = home_2subs_array_out[0]
                    home_dur_in_out[player_out][0] = 2
                    home_activity_count_all[player_out][0] = 0
                    home_pos_mean[player_out][0] = 0
                    home_pos_mean[player_out][1] = 0
                    

                for player_home in range(0, len(home_team_player)):
                    x = home_pos_mean[player_home][0]
                    y = home_pos_mean[player_home][1]
                    
                    if x!=0:
                        home_nonzero += 1
                        
                if home_nonzero != home_prev_count:
                    break
                
                # scaling home position
                temporary_pos_home = np.zeros(shape=(10,2), dtype=np.float)
                tmp_pos_i = 0
                for player_home in range(0, len(home_team_player)):
                    st_jersey_n = int(home_dict_reverse[player_home])
                    if home_pos_mean[player_home][0]!=0 and player_positionId['homeTeam'][st_jersey_n] != 1:
                        temporary_pos_home[tmp_pos_i][0] = home_pos_mean[player_home][0]
                        temporary_pos_home[tmp_pos_i][1] = home_pos_mean[player_home][1]
                        tmp_pos_i +=1
                        
                        
                STDx = np.std(temporary_pos_home[:,0])
                STDy = np.std(temporary_pos_home[:,1])

                temporary_pos_home[:,0] = scale_linear_data(temporary_pos_home[:,0], 105-STDx, STDx)
                temporary_pos_home[:,1] = scale_linear_data(temporary_pos_home[:,1], 68-STDy, STDy)
                
                tmp_pos_i = 0
                for player_home in range(0, len(home_team_player)):
                    st_jersey_n = int(home_dict_reverse[player_home])
                    if home_pos_mean[player_home][0]!=0 and player_positionId['homeTeam'][st_jersey_n] != 1:
                        home_pos_mean[player_home][0] = temporary_pos_home[tmp_pos_i][0]
                        home_pos_mean[player_home][1] = temporary_pos_home[tmp_pos_i][1]
                        tmp_pos_i +=1
                        

                for player_home in range(0, len(home_team_player)):
                    x = home_pos_mean[player_home][0]
                    y = home_pos_mean[player_home][1]
                    gk_state=0
                    
                    st_jersey_n = int(home_dict_reverse[player_home])
                    if player_positionId['homeTeam'][st_jersey_n] == 1:
                        gk_state=1
            
                    if x!=0:
                        home_team_role[player_home] = define_role(x, y, role_segment_coord, gk_state)
                    else:
                        home_team_role[player_home] = -1

                        
                away_subs_count_check = 0
                away_subs_count_id = 0
                away_2subs_array_in = []
                away_2subs_array_out = []

                for player_away in range(0, len(away_team_player)):


                    if away_activity_count_all[player_away][1] != 0 and away_dur_in_out[player_away][0]==1:

                        caluclate_avrg_pos(minutes_step, away_pos_mean, player_away, away_pos_count_minutes, 
                                               away_activity_count_minutes)

                        if away_act_nozero[0]-away_act_nozero[1]<0 and away_act_nozero[0]!=0 and away_activity_count_all[player_away][0] == 0:
                            away_pos_mean[player_away][0] = 0 
                            away_pos_mean[player_away][1] = 0
                            away_activity_count_all[player_away][1] = 0

                    elif away_activity_count_all[player_away][1] == 0 and away_dur_in_out[player_away][0]==1 and away_dur_in_out[player_away][4]==1:


                        # player current and whole game duration durng match
                        enter_half = away_dur_in_out[player_away][1]
                        enter_time = away_dur_in_out[player_away][2]
                        p_game_play_dur = away_dur_in_out[player_away][3]

                        h = match_half_t
                        f_h_d= first_half_duration

                        if enter_half == 1:
                            current_play_dur = math.floor( ((f_h_d - enter_time) + (i-45*60)*(h-1) )/60 )
                        elif enter_half == 2:
                            current_play_dur = math.floor( (i - enter_time)/60 )

                        if current_play_dur < p_game_play_dur:
                            caluclate_avrg_pos(minutes_step, away_pos_mean, player_away, away_pos_count_minutes, 
                                           away_activity_count_minutes)
                            away_activity_count_all[player_away][1] = 1

                        else:
                            away_dur_in_out[player_away][0] = 2
                            away_pos_mean[player_away][0] = 0 
                            away_pos_mean[player_away][1] = 0


                    elif away_activity_count_all[player_away][1] == 0 and away_dur_in_out[player_away][0]==1:

                        if away_act_nozero[0] == away_act_nozero[1]:
                            away_dur_in_out[player_away][0] = 2
                            away_pos_mean[player_away][0] = 0 
                            away_pos_mean[player_away][1] = 0


                        elif away_act_nozero[0] > away_act_nozero[1]:
                            caluclate_avrg_pos(minutes_step, away_pos_mean, player_away, away_pos_count_minutes, 
                                           away_activity_count_minutes)
                            away_activity_count_all[player_away][1] = 1
                            away_subs_count_check += 1
                            away_2subs_array_out.append(player_away)
                            away_subs_count_id = player_away

                    else:
                        away_pos_mean[player_away][0] = 0 
                        away_pos_mean[player_away][1] = 0


                for player_away in range(0, len(away_team_player)):
                    if away_subs_count_check >= 1 and (away_act_nozero[0]-away_act_nozero[1]==1):
                            d = caluclate_nzero_activity_count(minutes_step, 0, away_subs_count_id, away_activity_count_minutes)
                            if away_activity_count_all[player_away][0] ==0 and away_activity_count_all[player_away][1] == 1:
                                if d==14:
                                    d-=2
                                result = caluclate_nzero_activity_count(minutes_step, d+1, player_away, away_activity_count_minutes)
                                if result != -1:
                                    away_2subs_array_in.append(player_away)

                    away_activity_count_all[player_away][0] = away_activity_count_all[player_away][1]
                    away_activity_count_all[player_away][1] = 0


                # number of last non-zero activties
                away_last_nz_act_count = caluclate_last_nzero_ac_count(len(away_team_player), away_activity_count_all)

                if away_last_nz_act_count == prev_away_nonzero and len(away_2subs_array_out)==2:
                    for away_2subs_i in range(0,2):
                        player = away_2subs_array_out[away_2subs_i]
                        if away_activity_count_minutes[player][minutes_step-1] == 0:
                            away_dur_in_out[player][0] = 2
                            away_activity_count_all[player][0] = 0
                            away_pos_mean[player][0] = 0
                            away_pos_mean[player][1] = 0


                if len(away_2subs_array_out)>=2:
                    for away_2subs_i in range(0,2):
                        player = away_2subs_array_out[away_2subs_i]
                        away_activity_count_all[player][0] = 1

                if len(away_2subs_array_in) == 2 and away_subs_count_check==2:
                    for away_2subs_i in range(0,2):
                        player = away_2subs_array_out[away_2subs_i]
                        away_dur_in_out[player][0] = 2
                        away_activity_count_all[player][0] = 0
                        away_pos_mean[player][0] = 0
                        away_pos_mean[player][1] = 0

                if len(away_2subs_array_in) == 1 and away_subs_count_check>=2:
                    player = away_2subs_array_in[0]
                    away_activity_count_all[player][0] = 0
                    away_pos_mean[player][0] = 0
                    away_pos_mean[player][1] = 0

                if away_act_nozero[0]-away_act_nozero[1]==1 and away_subs_count_check==1 and len(away_2subs_array_in) == 1:
                    player_out = away_2subs_array_out[0]
                    away_dur_in_out[player_out][0] = 2
                    away_activity_count_all[player_out][0] = 0
                    away_pos_mean[player_out][0] = 0
                    away_pos_mean[player_out][1] = 0

                
                for player_away in range(0, len(away_team_player)):
                    x = away_pos_mean[player_away][0]
                    y = away_pos_mean[player_away][1]
                    
                    if x!=0:
                        away_nonzero += 1
                        
                if away_nonzero != away_prev_count:
                    break

                # converting away X position and scaling them

                temporary_pos_away = np.zeros(shape=(10,2), dtype=np.float)
                
                tmp_pos_i = 0
                for player_away in range(0, len(away_team_player)):
                    st_jersey_n = int(away_dict_reverse[player_away])
                    
                    if away_pos_mean[player_away][0]!=0:
                        away_pos_mean[player_away][0] = 105-away_pos_mean[player_away][0]
                        if player_positionId['awayTeam'][st_jersey_n] != 1:
                            temporary_pos_away[tmp_pos_i][0] = away_pos_mean[player_away][0]
                            temporary_pos_away[tmp_pos_i][1] = away_pos_mean[player_away][1]
                            tmp_pos_i +=1
                            
                
                STDx = np.std(temporary_pos_away[:,0])
                STDy = np.std(temporary_pos_away[:,1])

                temporary_pos_away[:,0] = scale_linear_data(temporary_pos_away[:,0], 105-STDx, STDx)
                temporary_pos_away[:,1] = scale_linear_data(temporary_pos_away[:,1], 68-STDy, STDy)
                
                tmp_pos_i = 0
                for player_away in range(0, len(away_team_player)):
                    st_jersey_n = int(away_dict_reverse[player_away])
                    if away_pos_mean[player_away][0]!=0 and player_positionId['awayTeam'][st_jersey_n] != 1:
                        away_pos_mean[player_away][0] = temporary_pos_away[tmp_pos_i][0]
                        away_pos_mean[player_away][1] = temporary_pos_away[tmp_pos_i][1]
                        tmp_pos_i +=1
                

                for player_away in range(0, len(away_team_player)):
                    x = away_pos_mean[player_away][0]
                    y = away_pos_mean[player_away][1]
                    
                    gk_state=0
                    
                    st_jersey_n = int(away_dict_reverse[player_away])
                    if player_positionId['awayTeam'][st_jersey_n] == 1:
                        gk_state=1
            
                    if x!=0:
                        away_team_role[player_away] = define_role(x, y, role_segment_coord, gk_state)
                    else:
                        away_team_role[player_away] = -1

                    
            elif (i-(match_half_t-1)*45*60)==threshold_second:
                
                for player_home in range(0, len(home_team_player)):
                    caluclate_avrg_pos(minutes_step, home_pos_mean, player_home, home_pos_count_minutes, 
                                           home_activity_count_minutes)
                    
                for player_away in range(0, len(away_team_player)):
                    caluclate_avrg_pos(minutes_step, away_pos_mean, player_away, away_pos_count_minutes, 
                                               away_activity_count_minutes)
                    
                    
                # converting away X position and scaling them

                temporary_pos_away = np.zeros(shape=(10,2), dtype=np.float)
                
                tmp_pos_i = 0
                for player_away in range(0, len(away_team_player)):
                    st_jersey_n = int(away_dict_reverse[player_away])
                    
                    if away_pos_mean[player_away][0]!=0:
                        away_pos_mean[player_away][0] = 105-away_pos_mean[player_away][0]
                        if player_positionId['awayTeam'][st_jersey_n] != 1:
                            temporary_pos_away[tmp_pos_i][0] = away_pos_mean[player_away][0]
                            temporary_pos_away[tmp_pos_i][1] = away_pos_mean[player_away][1]
                            tmp_pos_i +=1
                        

                STDx = np.std(temporary_pos_away[:,0])
                STDy = np.std(temporary_pos_away[:,1])        
                        
                temporary_pos_away[:,0] = scale_linear_data(temporary_pos_away[:,0], 105-STDx, STDx)
                temporary_pos_away[:,1] = scale_linear_data(temporary_pos_away[:,1], 68-STDx, STDy)
                
                tmp_pos_i = 0
                for player_away in range(0, len(away_team_player)):
                    st_jersey_n = int(away_dict_reverse[player_away])
                    if away_pos_mean[player_away][0]!=0 and player_positionId['awayTeam'][st_jersey_n] != 1:
                        away_pos_mean[player_away][0] = temporary_pos_away[tmp_pos_i][0]
                        away_pos_mean[player_away][1] = temporary_pos_away[tmp_pos_i][1]
                        tmp_pos_i +=1
                
                
                # scaling home position
                temporary_pos_home = np.zeros(shape=(10,2), dtype=np.float)
                tmp_pos_i = 0
                for player_home in range(0, len(home_team_player)):
                    st_jersey_n = int(home_dict_reverse[player_home])
                    if home_pos_mean[player_home][0]!=0 and player_positionId['homeTeam'][st_jersey_n] != 1:
                        temporary_pos_home[tmp_pos_i][0] = home_pos_mean[player_home][0]
                        temporary_pos_home[tmp_pos_i][1] = home_pos_mean[player_home][1]
                        tmp_pos_i +=1
                        
                    
                STDx = np.std(temporary_pos_home[:,0])
                STDy = np.std(temporary_pos_home[:,1]) 
                
                temporary_pos_home[:,0] = scale_linear_data(temporary_pos_home[:,0], 105-STDx, STDx)
                temporary_pos_home[:,1] = scale_linear_data(temporary_pos_home[:,1], 68-STDx, STDy)
                
                tmp_pos_i = 0
                for player_home in range(0, len(home_team_player)):
                    st_jersey_n = int(home_dict_reverse[player_home])
                    if home_pos_mean[player_home][0]!=0 and player_positionId['homeTeam'][st_jersey_n] != 1:
                        home_pos_mean[player_home][0] = temporary_pos_home[tmp_pos_i][0]
                        home_pos_mean[player_home][1] = temporary_pos_home[tmp_pos_i][1]
                        tmp_pos_i +=1
                
  
                for player_home in range(0, len(home_team_player)):
                     
                    x = home_pos_mean[player_home][0]
                    y = home_pos_mean[player_home][1]
                    
                    gk_state=0
                    
                    st_jersey_n = int(home_dict_reverse[player_home])
                    if player_positionId['homeTeam'][st_jersey_n] == 1:
                        gk_state=1
            
                    if x!=0:
                        home_team_role[player_home] = define_role(x, y, role_segment_coord,gk_state)
                    else:
                        home_team_role[player_home] = -1
                    
                
                for player_away in range(0, len(away_team_player)):
                    
                    x = away_pos_mean[player_away][0]
                    y = away_pos_mean[player_away][1]
                    
                    gk_state=0
                    
                    st_jersey_n = int(away_dict_reverse[player_away])
                    if player_positionId['awayTeam'][st_jersey_n] == 1:
                        gk_state=1

                        
                    if x!=0:
                        away_team_role[player_away] = define_role(x, y, role_segment_coord, gk_state)
                    else:
                        away_team_role[player_away] = -1

                        
            if (i-(match_half_t-1)*45*60)>=threshold_second:
                
                if tmp_t_2!=i:
                       
                    tmp_t_2=i
                    
                    features_adjust(home_wingers_fd, home_centers_fd, home_feature_count, home_f_speed_dict, home_x_data, home_y_data)
                    features_adjust(away_wingers_fd, away_centers_fd, away_feature_count, away_f_speed_dict, away_x_data, away_y_data)
                   
                    refree_fd = np.resize(refree_features,(5))
                    GK_home_fd = np.resize(GK_home_features,(5))
                    GK_away_fd = np.resize(GK_away_features,(5))

                    all_data_temp = np.concatenate((refree_fd,          # referee features
                                                    GK_home_fd,         # goalKepper features
                                                    home_wingers_fd[0], # homeLeftBack features
                                                    home_centers_fd[0], # homeCenterBack features
                                                    home_wingers_fd[1], # homeRightBack features
                                                    home_wingers_fd[2], # homeLeftWinger features
                                                    home_centers_fd[1], # homeCenterMiddle features
                                                    home_wingers_fd[3], # homeRightWinger features
                                                    home_centers_fd[2], # homeCenterForward features
                                                    
                                                    GK_away_fd,         # goalKepper features
                                                    away_wingers_fd[0], # awayLeftBack features
                                                    away_centers_fd[0], # awayCenterBack features
                                                    away_wingers_fd[1], # awayRightBack features
                                                    away_wingers_fd[2], # awayLeftWinger features
                                                    away_centers_fd[1], # awayCenterMiddle features
                                                    away_wingers_fd[3], # awayRightWinger features
                                                    away_centers_fd[2], # awayCenterForward features
                                                        
                                                    ball_pos_index
                                                   ), axis=0)
                    
                    all_data_temp = np.resize(all_data_temp,(1,len(all_data_temp)))
                    
                    additional_data = np.array([id, match_half_t, i//60, i%60])
                    
                    features_count_tmp = np.concatenate((additional_data, 
                                                  home_feature_count[0], # homeLeftBack counts
                                                  home_feature_count[4], # homeCenterBack counts
                                                  home_feature_count[1], # homeRightBack counts
                                                  home_feature_count[2], # homeLeftWinger counts
                                                  home_feature_count[5], # homeCenterMiddle counts
                                                  home_feature_count[3], # homeRightWinger counts
                                                  home_feature_count[6], # homeCenterForward counts

                                                  away_feature_count[0], # awayLeftBack counts
                                                  away_feature_count[4], # awayCenterBack counts
                                                  away_feature_count[1], # awayRightBack counts
                                                  away_feature_count[2], # awayLeftWinger counts
                                                  away_feature_count[5], # awayCenterMiddle counts
                                                  away_feature_count[3], # awayRightWinger counts
                                                  away_feature_count[6], # awayCenterForward counts
                                                  ), axis=0)
                    
                    features_count_tmp = np.resize(features_count_tmp,(1,len(features_count_tmp)))

                    all_data_count.append(features_count_tmp)
                    all_data.append(all_data_temp)

                    # then clear every thing
                    home_wingers_fd[:] = 0 
                    home_centers_fd[:] = 0
                    away_wingers_fd[:] = 0
                    away_centers_fd[:] = 0
                    
                    home_feature_count[:] = 0
                    away_feature_count[:] = 0
                    
                    refree_features[:] = 0
                    GK_home_features[:] = 0
                    GK_away_features[:] = 0
                    ball_pos_index[0] = 0
                    
                    home_f_speed_dict = {0:{}, 1:{}, 2:{}, 3:{}, 4:{}, 5:{}, 6:{}}
                    away_f_speed_dict = {0:{}, 1:{}, 2:{}, 3:{}, 4:{}, 5:{}, 6:{}}
                    
                    
                    for i in range(0, len(home_team_player)):
                        home_x_data[i] = (shift(home_x_data[i], -1, cval=0))
                        home_y_data[i] = (shift(home_y_data[i], -1, cval=0))
                    
                    for i in range(0, len(away_team_player)):
                        away_x_data[i] = (shift(away_x_data[i], -1, cval=0))
                        away_y_data[i] = (shift(away_y_data[i], -1, cval=0))
                        
                    referee_xy_data[0][0] = referee_xy_data[0][1]
                    referee_xy_data[0][1] = 0
                    
                    referee_xy_data[1][0] = referee_xy_data[1][1]
                    referee_xy_data[1][1] = 0
                
               
                
                if has_ball_teamId_t==0:
                    ball_pos_index[0] = 0
                   
                   

                # REFREEE:
                if team_id_t==0:
                    refree_features[0][0] = x_pos
                    refree_features[0][1] = y_pos
                    refree_features[0][2] = speed_t
                    
                    referee_xy_data[0][1] = x_pos
                    referee_xy_data[1][1] = y_pos
                    deltaX = referee_xy_data[0][1]-referee_xy_data[0][0]
                    deltaY = referee_xy_data[1][1]-referee_xy_data[1][0]
                    
                    refree_features[0][3] = define_direction(deltaX)
                    refree_features[0][4] = define_direction(deltaX)

                # HOME
                elif team_id_t == home_team_id: 
                    h_jersey_no = home_dict[str(jersey_number_t)]
                    r = int(home_team_role[h_jersey_no])
                    
                    if has_ball_teamId_t == home_team_id and has_ball_jerseyId_t == jersey_number_t:
                        ball_pos_index[0] = define_pitch_index(x_pos,y_pos)
                       

                    # Goalkepper
                    if r==10:
                        
                        GK_home_features[0][0] = x_pos
                        GK_home_features[0][1] = y_pos
                        GK_home_features[0][2] = speed_t
                        
                        home_x_data[h_jersey_no][1] = x_pos
                        home_y_data[h_jersey_no][1] = y_pos
                        deltaX = home_x_data[h_jersey_no][1]-home_x_data[h_jersey_no][0]
                        deltaY = home_y_data[h_jersey_no][1]-home_y_data[h_jersey_no][0]
                        
                        GK_home_features[0][3] = define_direction(deltaX)
                        GK_home_features[0][4] = define_direction(deltaY)
                        
        
                    #PLAYERS
                    elif r!=-1:
                        home_x_data[h_jersey_no][1] = x_pos
                        home_y_data[h_jersey_no][1] = y_pos
                        x1 = home_x_data[h_jersey_no][0]
                        feature_generation(0,r,x1,x_pos,y_pos,speed_t,h_jersey_no,home_wingers_fd, home_feature_count)
                        
                # AWAY        
                elif team_id_t == away_team_id: 
                    a_jersey_no = away_dict[str(jersey_number_t)]
                    r = int(away_team_role[a_jersey_no])
                    
                    if has_ball_teamId_t == away_team_id and has_ball_jerseyId_t== jersey_number_t:
                        ball_pos_index[0] = define_pitch_index(x_pos,y_pos)
                        
                    # Goalkepper
                    if r==10:
                        GK_away_features[0][0] = x_pos
                        GK_away_features[0][1] = y_pos
                        GK_away_features[0][2] = speed_t
                        
                        away_x_data[a_jersey_no][1] = x_pos
                        away_y_data[a_jersey_no][1] = y_pos
                        deltaX = away_x_data[a_jersey_no][1]-away_x_data[a_jersey_no][0]
                        deltaY = away_y_data[a_jersey_no][1]-away_y_data[a_jersey_no][0]
                        
                        GK_away_features[0][3] = define_direction(deltaX)
                        GK_away_features[0][4] = define_direction(deltaY)
                        
                    # PLAYERS
                    elif r!=-1:
                        away_x_data[a_jersey_no][1] = x_pos
                        away_y_data[a_jersey_no][1] = y_pos
                        x1 = away_x_data[a_jersey_no][0]
                        feature_generation(1,r,x1,x_pos,y_pos,speed_t,a_jersey_no,away_wingers_fd, away_feature_count)
    
        
        # writing players' role assignments count to csv file
        all_data_count = np.array(all_data_count)
        length_s_count, _, width_s_count = all_data_count.shape
        all_data_count = np.resize(all_data_count,(length_s_count,width_s_count))
        all_data_count = all_data_count.astype(int)

        all_data = np.array(all_data)
        length_f, _, width_f = all_data.shape
        all_data = np.resize(all_data,(length_f,width_f))
        
        np.savetxt("../data/match_" + str(id) + "/features_set.csv", all_data, delimiter=',', header = features_set_header)
        np.savetxt("../data/match_" + str(id) + "/role_assignment_count_v2.csv", all_data_count, delimiter=',', header = group_count_header)

        pd.options.display.float_format = '{:.0f}'.format 
        roleassign_df = pd.read_csv('../data/match_{}/role_assignment_count_v2.csv'.format(id))
        display(roleassign_df.tail())
        
        pd.options.display.float_format = '{:.5f}'.format 
        features_df = pd.read_csv('../data/match_{}/features_set.csv'.format(id))
        display(features_df.tail())


Match id:60565


,# matchId,half,min,sec,homeLeftBack,homeCenterBack,homeRightBack,homeLeftWinger,homeCenterMiddle,homeRightWinger,homeCenterForward,awayLeftBack,awayCenterBack,awayRightBack,awayLeftWinger,awayCenterMiddle,awayRightWinger,awayCenterForward
5416,60565,2,94,5,1,2,1,0,4,1,1,1,2,1,1,2,1,2
5417,60565,2,94,6,1,2,1,0,4,1,1,1,2,1,1,2,1,2
5418,60565,2,94,7,1,2,1,0,4,1,1,1,2,1,1,2,1,2
5419,60565,2,94,8,1,2,1,0,4,1,1,1,2,1,1,2,1,2
5420,60565,2,94,9,1,2,1,0,4,1,1,1,2,1,1,2,1,2


,# refX,refY,refSpeed,refDirectX,refDirectY,homeGoalKeeperX,homeGoalKeeperY,homeGoalKeeperSpeed,homeGoalKeeperDirectX,homeGoalKeeperDirectY,homeLeftBackAvgX,homeLeftBackAvgY,homeLeftBackDenisty,homeLeftBackAvgSpeed,homeLeftBackAvgDirectX,homeCenterBackSlowAvgX,homeCenterBackSlowAvgY,homeCenterBackSlowDenisity,homeCenterBackSlowAvgSpeed,homeCenterBackSlowAvgDirectX,homeCenterBackHirAvgX,homeCenterBackHirAvgY,homeCenterBackHirAvgSpeed,homeCenterBackHirAvgDirectX,homeCenterBackMaxSprintX,homeCenterBackMaxSprintY,homeCenterBackMaxSprintSpeed,homeCenterBackMaxSprintDirectX,homeCenterBackMaxSprintDirectY,homeRightBackAvgX,homeRightBackAvgY,homeRightBackDenisty,homeRightBackAvgSpeed,homeRightBackAvgDirectX,homeLeftWingerAvgX,homeLeftWingerAvgY,homeLeftWingerDenisty,homeLeftWingerAvgSpeed,homeLeftWingerAvgDirectX,homeCenterMiddleSlowAvgX,homeCenterMiddleSlowAvgY,homeCenterMiddleSlowDenisity,homeCenterMiddleSlowAvgSpeed,homeCenterMiddleSlowAvgDirectX,homeCenterMiddleHirAvgX,homeCenterMiddleHirAvgY,homeCenterMiddleHirAvgSpeed,homeCenterMiddleHirAvgDirectX,homeCenterMiddleMaxSprintX,homeCenterMiddleMaxSprintY,homeCenterMiddleMaxSprintSpeed,homeCenterMiddleMaxSprintDirectX,homeCenterMiddleMaxSprintDirectY,homeRightWingerAvgX,homeRightWingerAvgY,homeRightWingerDenisty,homeRightWingerAvgSpeed,homeRightWingerAvgDirectX,homeCenterForwardSlowAvgX,homeCenterForwardSlowAvgY,homeCenterForwardSlowDenisity,homeCenterForwardSlowAvgSpeed,homeCenterForwardSlowAvgDirectX,homeCenterForwardHirAvgX,homeCenterForwardHirAvgY,homeCenterForwardHirAvgSpeed,homeCenterForwardHirAvgDirectX,homeCenterForwardMaxSprintX,homeCenterForwardMaxSprintY,homeCenterForwardMaxSprintSpeed,homeCenterForwardMaxSprintDirectX,homeCenterForwardMaxSprintDirectY,awayGoalKeeperX,awayGoalKeeperY,awayGoalKeeperSpeed,awayGoalKeeperDirectX,awayGoalKeeperDirectY,awayLeftBackAvgX,awayLeftBackAvgY,awayLeftBackDenisty,awayLeftBackAvgSpeed,awayLeftBackAvgDirectX,awayCenterBackSlowAvgX,awayCenterBackSlowAvgY,awayCenterBackSlowDenisity,awayCenterBackSlowAvgSpeed,awayCenterBackSlowAvgDirectX,awayCenterBackHirAvgX,awayCenterBackHirAvgY,awayCenterBackHirAvgSpeed,awayCenterBackHirAvgDirectX,awayCenterBackMaxSprintX,awayCenterBackMaxSprintY,awayCenterBackMaxSprintSpeed,awayCenterBackMaxSprintDirectX,awayCenterBackMaxSprintDirectY,awayRightBackAvgX,awayRightBackAvgY,awayRightBackDenisty,awayRightBackAvgSpeed,awayRightBackAvgDirectX,awayLeftWingerAvgX,awayLeftWingerAvgY,awayLeftWingerDenisty,awayLeftWingerAvgSpeed,awayLeftWingerAvgDirectX,awayCenterMiddleSlowAvgX,awayCenterMiddleSlowAvgY,awayCenterMiddleSlowDenisity,awayCenterMiddleSlowAvgSpeed,awayCenterMiddleSlowAvgDirectX,awayCenterMiddleHirAvgX,awayCenterMiddleHirAvgY,awayCenterMiddleHirAvgSpeed,awayCenterMiddleHirAvgDirectX,awayCenterMiddleMaxSprintX,awayCenterMiddleMaxSprintY,awayCenterMiddleMaxSprintSpeed,awayCenterMiddleMaxSprintDirectX,awayCenterMiddleMaxSprintDirectY,awayRightWingerAvgX,awayRightWingerAvgY,awayRightWingerDenisty,awayRightWingerAvgSpeed,awayRightWingerAvgDirectX,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardSlowDenisity,awayCenterForwardSlowAvgSpeed,awayCenterForwardSlowAvgDirectX,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardHirAvgSpeed,awayCenterForwardHirAvgDirectX,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,awayCenterForwardMaxSprintDirectX,awayCenterForwardMaxSprintDirectY,pitch_index
5416,25.49000,47.74000,0.53000,-1.00000,-1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,19.08000,27.72000,1.00000,0.70000,-1.00000,15.87000,34.98000,1.00000,1.09000,0.35000,15.32000,41.25000,1.52000,1.16000,0.00000,0.00000,0.00000,0.00000,0.00000,26.18000,56.31000,1.00000,2.24000,-1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,26.74333,36.77667,0.04907,0.73000,-0.46333,36.09000,42.89000,1.99000,-0.87000,0.00000,0.00000,0.00000,0.00000,0.00000,15.66000,57.44000,1.00000,3.14000,-1.00000,49.45000,44.63000,1.00000,1.46000,-1.50000,0.00000,0.00000,0.00000,0